In [1]:
from mtcnn import MTCNN
from PIL import Image
import glob
from tensorflow.keras.models import *
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.preprocessing import image
from matplotlib import pyplot as plt

In [2]:
model_age = load_model("Age/age_best_model.h5")
model_gender = load_model("Gender/age_best_model.h5")
model_FER_emotion = load_model("FER_Emotion/FER_VGG16.h5")
model_CK_emotion = load_model("CK_Emotion/v16.h5")
detector = MTCNN()

In [3]:
def rgb2gray(rgb):
    r, g, b = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def detect_face(img):
    mt_res = detector.detect_faces(img)
    return_res = []

    for face in mt_res:
        x, y, width, height = face['box']
        center = [x + (width / 2), y + (height / 2)]
        max_border = max(width, height)

        # center alignment
        left = max(int(center[0] - (max_border / 2)), 0)
        right = max(int(center[0] + (max_border / 2)), 0)
        top = max(int(center[1] - (max_border / 2)), 0)
        bottom = max(int(center[1] + (max_border / 2)), 0)

        # crop the face
        center_img_k = img[top:top + max_border,
                       left:left + max_border, :]
        fer_center_img = np.array(Image.fromarray(center_img_k).resize([64, 64]))
        ck_center_img = np.array(Image.fromarray(center_img_k).resize([128, 128]))
        center_img = np.array(Image.fromarray(center_img_k).resize([224, 224]))

        # print("center_img--------------", center_img)

        # create predictions
        #         sex_preds = sex_model.predict(center_img.reshape(1,224,224,3))[0][0]
        #         age_preds = age_model.predict(center_img.reshape(1,224,224,3))[0][0]

        # FER - MODEL
        fer_img = image.img_to_array(fer_center_img)
        fer_img = fer_img / 255
        fer_img = np.expand_dims(fer_img, axis=0)
        fer_emotion_preds = model_FER_emotion.predict(fer_img)

        # CK - MODEL
        ck_img = image.img_to_array(ck_center_img)
        ck_img = ck_img / 255
        ck_img = np.expand_dims(ck_img, axis=0)
        ck_emotion_preds = model_CK_emotion.predict(ck_img)

        # Age/Gender - Model
        age_gender_img = image.img_to_array(center_img)
        age_gender_img = age_gender_img / 255
        age_gender_img = np.expand_dims(age_gender_img, axis=0)
        age_preds = model_age.predict(age_gender_img)
        gender_preds = model_gender.predict(age_gender_img)
        # output to the cv2
        return_res.append([top, right, bottom, left, fer_emotion_preds, ck_emotion_preds, age_preds, gender_preds])

    return return_res

In [4]:
# Get a reference to webcam
video_capture = cv2.VideoCapture(0)
# print("video_capture--------------",video_capture)

emotion_dict_ck = {
    0: 'anger',
    1: 'contempt',
    2: 'disgust',
    3: 'fear',
    4: 'happy',
    5: 'sadness',
    6: 'surprise'
}

emotion_dict_fer = {
    0: 'anger',
    1: 'disgust',
    2: 'fear',
    3: 'happy',
    4: 'neutral',
    5: 'sad',
    6: 'surprise'
}

categories = {
    0: '0-20',
    1: '20-25',
    2: '25-30',
    3: '30-40',
    4: '40-60',
    5: '60+'
}
gender_categories = {
    0: 'Male',
    1: 'Female'
}

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    # print("--------------", frame.shape)
    # Convert the image from BGR color (which OpenCV uses) to RGB color
    rgb_frame = frame[:, :, ::-1]
    # print("rgb_frame--------------", rgb_frame.shape)
    # Find all the faces in the current frame of video
    face_locations = detect_face(rgb_frame)
    # print("face_locations--------------", face_locations)

    # Display the results
    for top, right, bottom, left, fer_emotion_preds, ck_emotion_preds, age_preds, gender_preds in face_locations:
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        cv2.putText(frame, 'FER Emotion: {}({:.3f})'.format(emotion_dict_fer[np.argmax(fer_emotion_preds)], np.max(fer_emotion_preds)),
                    (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 1)

        cv2.putText(frame, 'CK Emotion: {}({:.3f})'.format(emotion_dict_ck[np.argmax(ck_emotion_preds)], np.max(ck_emotion_preds)),
                    (left, top - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 1)

        cv2.putText(frame, 'Age: {}({:.3f})'.format(categories[np.argmax(age_preds)], np.max(age_preds)),
                    (left, top - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 1)

        cv2.putText(frame, 'Gender: {}({:.3f})'.format(gender_categories[np.argmax(gender_preds)], np.max(gender_preds)),
                    (left, top - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 107ms/step


1/1 [==============================] - 0s 19ms/step


...

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 170ms/step


1/1 [==============================] - 0s 266ms/step


In [5]:
# import tensorflow as tf
# gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
# session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# # tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
